In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from apps.environment_simulator.models import SimulatedCommodity, SimulatedCommodityCovariance, SimulatedCommodityBuffer

import numpy as np
import json
from datetime import datetime

In [2]:
commodities = {
    "crude_oil": {
        "price": 95,
        "index": 1,
        "sd": 2,
        "min_price": 12,
        "max_price": 300,
        "avg_forward_steps": 10,
        "avg_backward_steps": 10,
        "unit": "barrel"
    },
    "natural_gas": {
        "price": 2.74,
        "index": 2,
        "sd": 0.03,
        "min_price": 0.5,
        "max_price": 6,
        "avg_forward_steps": 5,
        "avg_backward_steps": 5,
        "unit": "barrel"
    },
    "al_com": {
        "price": 319,
        "index": 3,
        "sd": 3,
        "min_price": 30,
        "max_price": 800,
        "avg_forward_steps": 7,
        "avg_backward_steps": 7,
        "unit": "kg"
    },
    "copper": {
        "price": 8030,
        "index": 4,
        "sd": 15,
        "min_price": 4000,
        "max_price": 12000,
        "avg_forward_steps": 15,
        "avg_backward_steps": 15,
        "unit": "kg"

    },
    "nickel_lme": {
        "price": 25581,
        "index": 5,
        "sd": 30,
        "min_price": 15000,
        "max_price": 40000,
        "avg_forward_steps": 3,
        "avg_backward_steps": 3,
        "unit": "kg"

    },
    "lead_lme": {
        "price": 2110,
        "index": 6,
        "sd": 12,
        "min_price": 300,
        "max_price": 7000,
        "avg_forward_steps": 9,
        "avg_backward_steps": 9,
        "unit": "kg"

    # },
    # "zinc_lme": {
    #     "price": 3008,
    #     "index": 7,
    #     "sd": 15,
    #     "min_price": 500,
    #     "max_price": 9000,
    #     "avg_forward_steps": 4,
    #     "avg_backward_steps": 4,
    #     "unit": "kg"

    # },
    # "tin_lme": {
    #     "price": 21631,
    #     "index": 8,
    #     "sd": 75,
    #     "min_price": 5000,
    #     "max_price": 35000,
    #     "avg_forward_steps": 11,
    #     "avg_backward_steps": 11,
    #     "unit": "kg"

    # },
    # "iron_ore": {
    #     "price": 99.8,
    #     "index": 9,
    #     "sd": 1,
    #     "min_price": 8,
    #     "max_price": 500,
    #     "avg_forward_steps": 6,
    #     "avg_backward_steps": 6,
    #     "unit": "kg"

    # },
    # "hot_rolled_steel": {
    #     "price": 695,
    #     "index": 10,
    #     "sd": 6,
    #     "min_price": 100,
    #     "max_price": 3000,
    #     "avg_forward_steps": 4,
    #     "avg_backward_steps": 4,
    #     "unit": "kg"

    # },
    # "molybdeum": {
    #     "price": 37394,
    #     "index": 11,
    #     "sd": 120,
    #     "min_price": 8000,
    #     "max_price": 80000,
    #     "avg_forward_steps": 9,
    #     "avg_backward_steps": 9,
    #     "unit": "kg"

    # },
    # "cobalt": {
    #     "price": 51504,
    #     "index": 12,
    #     "sd": 200,
    #     "min_price": 12000,
    #     "max_price": 120000,
    #     "avg_forward_steps": 12,
    #     "avg_backward_steps": 12,
    #     "unit": "kg"

    # },
    # "al_lme": {
    #     "price": 2336,
    #     "index": 13,
    #     "sd": 28,
    #     "min_price": 120,
    #     "max_price": 11000,
    #     "avg_forward_steps": 7,
    #     "avg_backward_steps": 7,
    #     "unit": "kg"

    # },
    # "silicon": {
    #     "price": 3000,
    #     "index": 14,
    #     "sd": 10,
    #     "min_price": 700,
    #     "max_price": 15000,
    #     "avg_forward_steps": 8,
    #     "avg_backward_steps": 8,
    #     "unit": "kg"

    # },
    # "gold": {
    #     "price": 61000,
    #     "index": 15,
    #     "sd": 300,
    #     "min_price": 55000,
    #     "max_price": 200000,
    #     "avg_forward_steps": 13,
    #     "avg_backward_steps": 13,
    #     "unit": "kg"
    }
}
for a_cmmdty_name, a_cmmdty_data in commodities.items():
    sd = a_cmmdty_data["sd"]
    avg_price = (a_cmmdty_data["min_price"] + a_cmmdty_data["max_price"])/2
    grad_temp = avg_price/100
    pos_gradient = np.random.normal(grad_temp, sd)
    gradient = np.random.choice([pos_gradient, -1*pos_gradient])
    a_cmmdty_data["gradient"] = gradient

In [3]:
SimulatedCommodity.objects.all().delete()
sim_cmmdts = []
for cmmdty_name, cmmdty_data in commodities.items():
    new_cmmdty = SimulatedCommodity(
        name = cmmdty_name,
        index = cmmdty_data["index"],
        gradient = cmmdty_data["gradient"],
        sd = cmmdty_data["sd"],
        min_price = cmmdty_data["min_price"],
        max_price = cmmdty_data["max_price"],
        avg_forward_steps = cmmdty_data["avg_forward_steps"],
        avg_backward_steps = cmmdty_data["avg_backward_steps"],
        steps_left = cmmdty_data["avg_forward_steps"],
        units = cmmdty_data["unit"],
    )
    sim_cmmdts.append(new_cmmdty)

SimulatedCommodity.objects.bulk_create(sim_cmmdts)

[<SimulatedCommodity: 337, 1, crude_oil>,
 <SimulatedCommodity: 338, 2, natural_gas>,
 <SimulatedCommodity: 339, 3, al_com>,
 <SimulatedCommodity: 340, 4, copper>,
 <SimulatedCommodity: 341, 5, nickel_lme>,
 <SimulatedCommodity: 342, 6, lead_lme>]

In [4]:
np.random.seed(5)


SimulatedCommodityCovariance.objects.all().delete()
comvars = []
accounted_for_cmmdties = {}
new_commodities = SimulatedCommodity.objects.all()
for nw_cmmdty in new_commodities:
    # print("nw_cmmdty.id", nw_cmmdty.id)
    # print("recheck", json.dumps(accounted_for_cmmdties, indent=3))
    if nw_cmmdty.id not in accounted_for_cmmdties:
        accounted_for_cmmdties[nw_cmmdty.id] = {}
    cmmdty_a_cmmdties = accounted_for_cmmdties[nw_cmmdty.id]
    for a_cmmdty in new_commodities:
        if a_cmmdty.id != nw_cmmdty.id:
            if a_cmmdty.id not in accounted_for_cmmdties:
                accounted_for_cmmdties[a_cmmdty.id] = {}
            cmmdty_b_cmmdties = accounted_for_cmmdties[a_cmmdty.id]
            if (
                a_cmmdty.id not in cmmdty_a_cmmdties
                and nw_cmmdty.id not in cmmdty_b_cmmdties
            ):
                cmmdty_a_cmmdties[a_cmmdty.id] = a_cmmdty.id
                cmmdty_b_cmmdties[nw_cmmdty.id] = nw_cmmdty.id
                new_covar = SimulatedCommodityCovariance(
                    commodity_a = nw_cmmdty,
                    commodity_b = a_cmmdty,
                    factor = np.random.rand()*np.random.choice([0.5,-0.5])
                )
                comvars.append(new_covar)
                # print(json.dumps(accounted_for_cmmdties, indent=3))
                # print("")

SimulatedCommodityCovariance.objects.bulk_create(comvars)


[<SimulatedCommodityCovariance: 2641, crude_oil, natural_gas>,
 <SimulatedCommodityCovariance: 2642, crude_oil, al_com>,
 <SimulatedCommodityCovariance: 2643, crude_oil, copper>,
 <SimulatedCommodityCovariance: 2644, crude_oil, nickel_lme>,
 <SimulatedCommodityCovariance: 2645, crude_oil, lead_lme>,
 <SimulatedCommodityCovariance: 2646, natural_gas, al_com>,
 <SimulatedCommodityCovariance: 2647, natural_gas, copper>,
 <SimulatedCommodityCovariance: 2648, natural_gas, nickel_lme>,
 <SimulatedCommodityCovariance: 2649, natural_gas, lead_lme>,
 <SimulatedCommodityCovariance: 2650, al_com, copper>,
 <SimulatedCommodityCovariance: 2651, al_com, nickel_lme>,
 <SimulatedCommodityCovariance: 2652, al_com, lead_lme>,
 <SimulatedCommodityCovariance: 2653, copper, nickel_lme>,
 <SimulatedCommodityCovariance: 2654, copper, lead_lme>,
 <SimulatedCommodityCovariance: 2655, nickel_lme, lead_lme>]

In [5]:
def generate_first_buffer():
    starting_time_step = datetime(year=1995, month=1, day=1, hour=10)

    SimulatedCommodityBuffer.objects.all().delete()
    z_commodities = SimulatedCommodity.objects.all()
    buffer = []
    for z_cmmdty in z_commodities:
        mid_price = (z_cmmdty.min_price + z_cmmdty.max_price)
        mid_price = mid_price/2
        starting_snapshot = SimulatedCommodityBuffer(
            commodity = z_cmmdty,
            captured_at = starting_time_step,
            price_snapshot = mid_price
        )
        buffer.append(starting_snapshot)

    SimulatedCommodityBuffer.objects.bulk_create(buffer)
generate_first_buffer()

/Users/mirbilal/Desktop/MobCommission/commissionV2/v2env/lib/python3.10/site-packages/django/db/models/fields/__init__.py:1595: RuntimeWarning: DateTimeField SimulatedCommodityBuffer.captured_at received a naive datetime (1995-01-01 10:00:00) while time zone support is active.
  warnings.warn(


In [6]:
updtd_cmmdties = []
u_commodities = SimulatedCommodity.objects.all()
for u_cmmdty in u_commodities:
    u_cmmdty.steps_left = u_cmmdty.avg_forward_steps
    updtd_cmmdties.append(u_cmmdty)

u_commodities.bulk_update(updtd_cmmdties, ["steps_left"])

6

In [7]:
SimulatedCommodityBuffer.objects.all().delete()
generate_first_buffer()

In [46]:
def reset_covariances():
    updtd_covs = []
    covs = SimulatedCommodityCovariance.objects.all()
    for acov in covs:
        acov.factor = 0
        updtd_covs.append(acov)

    SimulatedCommodityCovariance.objects.bulk_update(updtd_covs, ["factor",])

reset_covariances()
